This document provides instructions for the installation and running of WRF version 4.3

If you have questions, feel free to contact Christopher Phillips, chris.phillips@uah.edu

Another useful source is the WRF user manual: https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/contents.html

Note that if you are running WRF on the UAH Matrix system you need to use the slurm system to issue the geogrid.exe, ungrib.exe, metgrid.exe, real.exe and wrf.exe commands. Exapmle slurm scripts are provided at the end of this guide.

--------------------
# --- Installing WRF ---


Prior to installing WRF, you will need the following modules loaded:

    module load null
    module load netcdf/4.4.0_mpi-pgi16.3_gcc
    module load ncarg/6.3_mpi-pgi16.3_gcc
    module load hdf5/v1.8.17_mpi-pgi16.3_gcc
    module load pgi/16.3
    module load mpich/3.2-pgi16.3

Additionally, you will need the following variables set:

    setenv WRF_EM_CORE 1
    setenv WRF_NMM_CORE 0
    setenv WRFIO_NCD_LARGE_FILE_SUPPORT 1
    setenv ENVCOMPDEFS -DWRF_CHEM
    setenv WRF_CHEM 0
    setenv WRF_KPP 0

    setenv JASPERLIB /common/pkgs/jasper/1.900.1-pgi12.10-EL6_64-201212201430/lib
    setenv JASPERINC /common/pkgs/jasper/1.900.1-pgi12.10-EL6_64-201212201430/include

Note that if you want to use chemistry, set WRF_CHEM to 1 instead of 0. It is recommended to place the module and setenv statements into the .tcshrc.matrix file located in your home directory.

While logged into Matrix, create a new directory for the wrf system and download the latest version
    
    mkdir wrf_model
    cd wrf_model
    git clone https://github.com/wrf-model/WRF

Then cd into the new WRF directory and run the configuration script

    ./configure

Select option 3 for parallel processing using the PGI compilers. When prompted, select no nesting. At this point, the following message should appear

    Testing for NetCDF, C and Fortran compiler

    This installation of NetCDF is 64-bit
        C compiler is 64-bit
        Fortran compiler is 64-bit
        It will build in 64-bit

Now open the newly created configure.wrf file

    vi configure.wrf

And make the following changes

1) SFC = mpifort

2) SCC = h5pcc

3) h5pcc

4) DM_FC = mpifort

5) DM_CC = h5pcc -DMPI2_SUPPORT

6) Add -Kieee to the FCOPTIM variable. Do NOT delete any other flags.

    FCOPTIM = -O3 -Kieee #-fastsse -Mvect=noaltcode -Msmartalloc -Mprefetch=distance:8 -Mfprelaxed # -Minfo=all =Mneginfo=all

Next, run the compilation script and wait. This can take anywhere from 30 minutes to 1.5 hours.

    ./compile em_real >& compile_real.log

After compilation, check the log file for successful build of the WRF executables.

    tail compile_scm.log

You should see something similar to the below:

    --->                  Executables successfully built                  <---

    -rwxr-x--- 1 cphillip cphillip 131990352 Oct 19 15:13 main/real.exe
    -rwxr-x--- 1 cphillip cphillip 159422160 Oct 19 15:13 main/wrf.exe

If so, congratulations, you have compile the WRF. If not, then check the log file for errors (searching for "severe" is the easiest way) and correct them.

Next, we will install the WRF Pre-processing system (WPS).

-----------------------------
#--- Installing the WPS ---

The WPS is the system used to prepare analysis data from other models such as the GFS or HRRR for input into the WRF model. The details for running the WPS are in a later section.

Change directories again to the wrf43 directory


```
cd ../
```

And clone the WPS directory.

```
git clone https://github.com/wrf-model/WPS
cd WPS
```

Similar to the WRF model, run the configuration script:

```
./configure
```

and select option 7 when prompted. You should see similar output:

```
------------------------------------------------------------------------
Configuration successful. To build the WPS, type: compile
------------------------------------------------------------------------

Testing for NetCDF, C and Fortran compiler

This installation NetCDF is 64-bit
C compiler is 64-bit
Fortran compiler is 64-bit
```

When configuration is complete make the following changes to configure.wps:

1) SFC             =       mpifort

2) SCC             =       h5pcc

3) DM_FC           =       mpifort

4) DM_CC           =       h5pcc -DMPI2_SUPPORT



Then compile.

```
./compile >& compile_wps.log
```

Unlike WRF, there is no clear completion message at the end of the log file, but ```ls -l``` should show links to 3 exectuables:

```
geogrid.exe -> geogrid/src/geogrid.exe
metgrid.exe -> metgrid/src/metgrid.exe
ungrib.exe -> ungrib/src/ungrib.exe
```

If these are not created, check the log files for error and re-compile. You can run ```./clean``` to clear out the intermediate compilation files between compile attempts.

Congratulations, you have now compiled both the WRF model and the WPS system.

-------------------------------------------------------
# --- Downloading Static Fields ---
The next step is to download and unpack the so-called- static input fields. These largely consist of terrian and topography data sets. We'll place them in the model root directory, wrf_model.

```
cd ..
mkdir static_data
wget https://www2.mmm.ucar.edu/wrf/src/wps_files/geog_high_res_mandatory.tar.gz
tar -xvzf geog_high_res_mandatory.tar.gz
```

This will likely take a while as there are many files in this data set. Once it is complete. You're ready to prepare your first simulation.

--------------------
#--- Setting Up Your First WRF Simulation ---

Running a simulation is reasonably straight-forward. The steps are:

1. Determine your domain
2. Download suitable data for boundary conditions.
3. Create the grid for your model and grid the input data using WPS.
4. Run real.exe to convert the WPS files into WRF input files.
5. Run wrf.exe.
6. Bask in the radiance of your completed WRF simulation

Each of these steps though has sub-parts and questions that you must give thought to, and the answers to those questions vary depending on the goals of your simulation. Weather modeling is both art and science.

# Determine your domain

We'll start with an exciting case. Let's simulate the December 13th, 2022 tornado outbreak over Lousiana and Mississippi. Some questions to consider:

- How large should the domain be?
- What grid spacing should we use?
- How many nests should there be?

As a rule of thumb, the domain should be large enough that features on the edges don't propagate into the center. Model edges can be weird becase data is interpolated from a larger model to the WRF grid. To compute this, we use:

$ Domain~Width~(km) = \frac{10~(m/s)~*~Simulation~Time~(s)}{1000 ~(m/km)} * 2$

Those who have taken a mid-latitude synoptic meteorology class might recognize 10 m/s as the scale speed of the zonal wind. Meridinal flow is typically about 5 m/s, so you can make the model height about half the width.

*NOTE: These are first-guesses only. Grid parameters can be optimized well beyond these rules.*

For a tornado outbreak, convection will be of prime-imprtance, so we need a grid spacing capable of resolving convection. We'll set grid spacing to 1 km. To get down from GFS's low resolution of 0.25 deg. We'll use three nests. The outer domain will be 9 km spacing, the second 3 km, and the innermost 1 km. Note the ratio between grids. WRF likes ratios of 3 and 5. These values provide a better interpolation at the nest edges. An outermost grid spacing of 9km is also about 3 times greater than GFS's native resolution, preserving this pattern even at the outermost edge.

# Download GFS analysis for boundary conditions
The historical GFS archive can be accessed through: https://rda.ucar.edu/datasets/ds084-1/

You will need an account, but they are free.

Located and download the following files. I suggest placing them inside an "input" directory within your WPS directory for now.
- gfs.0p25.2022121300.f000.grib2
- gfs.0p25.2022121300.f003.grib2
- gfs.0p25.2022121300.f006.grib2
- gfs.0p25.2022121300.f009.grib2
- gfs.0p25.2022121300.f012.grib2
- gfs.0p25.2022121300.f015.grib2
- gfs.0p25.2022121300.f018.grib2
- gfs.0p25.2022121300.f021.grib2
- gfs.0p25.2022121300.f024.grib2

These are 24 hours worth of GFS forecast output that will be used to supply the boundary conditoins for the WRF simulation. The next step is to run the WPS.

-----------------------
# --- Running WPS ---

Running the WPS consists of the following steps:

1. Create the namelist.wps file
2. Link the proper VTable file
3. Link the input data
4. Run geogrid
5. Run ungrib
6. Run metgrid

It can be noted that geogrid is its own program that only depends on the namelist.wps file, so it can be run anytime after creating the namelist but before running metgrid.

Create a document in the WPS directory and name it "namelist.wps"

Enter the following information except for anything after a "#." Those are my comments. In the future, the WRFDomainWizard is an excellent tool for creating your grids. https://esrl.noaa.gov/gsd/wrfportal/DomainWizardForLAPS.html

```
&share
 wrf_core = 'ARW',            # Which dynamical core to use. ARW is suitable for nearly every task.
 max_dom = 3,                 # The number of domains
 start_date = '2022-12-13_00:00:00', '2022-12-13_00:00:00', '2022-12-13_00:00:00' # Start and end dates, inclusive.
 end_date = '2022-12-14_00:00:00', '2022-12-14_00:00:00', '2022-12-14_00:00:00'   # One for each domain.
 interval_seconds = 10800     # How often to read a boundary file. GFS outputs every 3 hours (10800 seconds).
/

&geogrid
 parent_id         =   1,   1,   2,   # Which grid the nest belongs to.
 parent_grid_ratio =   1,   3,   3,   # The ratio to use for nest's grid spacing
 i_parent_start    =   1,  60, 83        # The grid coordinate that the nest starts at
 j_parent_start    =   1,  50, 84,
 e_we              =  200, 241, 226,       # How many points are in the grid.
 e_sn              =  180, 244, 229,       # Nest dimensions should be one more than a multiple of the ratio.
 geog_data_res = 'default','default', # The resolution of the static data to read. Default is fine.
 dx = 9000,                          # Grid spacing of the outermost grid
 dy = 9000,
 map_proj = 'lambert',                # Map projection. Lambert for mid-lats, mercator for tropics, polar for poles.
 ref_lat   =  31.784,                  # Grid center
 ref_lon   = -91.387,
 truelat1  =  31.784,                   # Lats/lons where a glob and the projection are "1 to 1" so to speak.
 truelat2  =  31.784,
 stand_lon = -91.387,                   # Keep the same as your center longitude.
 geog_data_path = '/path_to_static_fields/WPS_GEOG/'
/

&ungrib
 out_format = 'WPS'
 prefix = 'FILE'
/

&metgrid
 fg_name = 'FILE'
/

```

Once the namelist.wps file is created, link the vtable using the following command:
```
ln -sf ungrib/Variable_Tables/Vtable.GFS ./Vtable
```

You can check the link by running ls -l inside your WPS directory.
```
ls -l
Vtable -> ungrib/Variable_Tables/Vtable.GFS
```

Now inside your WPS directory, link the GFS input by running the following command:
```
./link_grib.csh input/gfs.*
```
A quick ls should show the GFS files linked into the WPS directory.
Next run geogrid, ungrib, and metgrid.
```
./geogrid.exe > geogrid.log
./ungrib.exe > ungrib.log
./metgrid > metgrid.log
```

At each step, ls should reveal new files.
Geogrid creates "geo_em" files. Ungrib creates "FILE" files. Metgrid creates "met_em" files.

"met_em" files are what you feed into the WRF real.exe and are the ultimate goal of running the WPS system.

-------------------------
#--- Running WRF ---

With the WPS finished, running WRF is fairly straight-forward, and consists of only a few steps.

1. Create the WRF namelist
2. Link the met_em files
3. Run real.exe to create wrf_input files
4. Run wrf.exe to create that sweet, sweet output.

Creating the namelist is a simple task because it is based on the WPS namelist. cd into the WRF/run directory.. The run directory is where you will run WRF and create the various namelist and input files. Open the namelist.input file. The namelist is a long file, so I'm only going to list the variables that you should check/change from the template. For more information on the available variables and what they do, see chapter 5 of the WRF user guide. Again, don't add comments behind a "#" symbol.

```
&time_control
  run_days = 0,
  run_hours = 24,
  run_minutes = 0,
  run_seconds = 0,
  start_year = 2022, 2022, 2022,
  start_month = 12, 12, 12,
  start_day = 13, 13, 13,
  start_hour = 0, 0, 0,
  start_minute = 0, 0, 0,
  start_second = 0, 0, 0,
  end_year = 2022, 2022, 2022,
  end_month = 12, 12, 12,
  end_day = 14, 14, 14,
  end_hour = 0, 0, 0,
  end_minute = 0, 0, 0,
  end_second = 0, 0, 0,
  interval_seconds = 10800
  history_interval = 180, 60, 15, # This is the output interval in minutes for each grid.
  frames_per_outfile = 1, 1, 1,   # This is how many timesteps to put in one file. Do one. It's easier all around.
/

&domains
  time_step = 54, # Model dt in seconds. Should divide evenly into history_interval.
  max_dom = 3,    # Number of grids. Often set to 1 (outer domain only) for testing then reset for the final run.
  e_we = 200, 241, 226, # Note these match namelist.wps
  e_sn = 180, 244, 229,
  e_vert = 45, 45, 45,  # The number of vertical levels in the model. Not in namelist.wps
  num_metgrid_levels = 34, # How many levels are in the met_em files. Changes depending on source data.
  dx = 9000, 3000, 1000,
  dy = 9000, 3000, 1000,
  grid_id = 1, 2, 3,
  parent_id = 1, 1, 2,
  i_parent_start = 1, 60, 83,    # What grid point to start the nest
  j_parent_start = 1, 50, 84,
  parent_grid_ratio = 1, 3, 3,
  parent_time_Step_ratio = 1, 3, 3, # Keep in sync with the grid_ratio
  feedback = 1, # If 1, nested grids will provide input to the outergrid. This is usually good.
/

&physics
  mp_physics =  8, 8, 8
  cu_physics = 1, 0, 0,
  ra_lw_physics = 4, 4, 4,
  ra_sw_physics = 4, 4, 4,
  bl_pbl_physics = 2, 2, 2,
  do_radar_ref = 1, # This tells the model to output simulated refl. It's slow, but nice for convective studies.
/

```

There are many, many more options. Many aren't even in the template. If you plan to use WRF regularly, give chapter 5 of the user manual a read through. There's a lot of valuable information there. Namelist settings are a blend of art and science. Play around to see what happens.

The next step is to link the met_em files.
```
ln -sf ../WPS/met_em* ./
```

Then run real.exe
```
real.exe
```

This should create files similar to the following:
```
ls -l wrfinput_*
wrfinput_d01
wrfinput_d02
wrfinput_d03
```

With the input files created, run wrf.exe. This will create output files and rsl log files.

That's it. That's how you run WRF. Welcome to modeling world where the data is imaginary but we use it anyway.


# SAMPLE SLURM SCRIPTS FOR MATRIX

For WPS

```
 #!/bin/bash
 ###Slurm Batch Script

 ###Email Address
 #SBATCH --mail-user=your_email@uah.edu

 ###Job Name
 #SBATCH -J mkgrid

 ### Partition (queue), select shared or standard
 #SBATCH -p shared

 ###TOTAL processors
 #SBATCH --ntasks 1

 ###Total run time estimate (D-HH:MM)
 #SBATCH -t 2-00:00


 ### memory (GB per CPU)
 #SBATCH --mem-per-cpu=12G


 ### Mail to user on an event
 ### common options are FAIL, BEGIN, END, REQUEUE
 #SBATCH --mail-type=BEGIN,END,FAIL

 ### Output files
 #SBATCH -o slurm-%j.out #STDOUT
 #SBATCH -e slurm-%j.err #STDERR

 cd your_wps_dir
 ./geogrid.exe > geogrid.log
 ./link_grib.csh input/gfs.*
 ./ungrib.exe > ungrib.log
 ./metgrid.exe > metgrid.log

```

For WRF

Run real.exe and wrf.exe separately since they both use the rsl log files.

```
 #!/bin/bash
 ###Slurm Batch Script

 ###Email Address
 #SBATCH --mail-user=your_email@uah.edu

 ###Job Name
 #SBATCH -J WRFsim

 ### Partition (queue), select shared or standard
 #SBATCH -p standard

 ###TOTAL processors
 #SBATCH --ntasks 48

 ###Total run time estimate (D-HH:MM)
 #SBATCH -t 7-00:00

 ### memory (GB per CPU)
 #SBATCH --mem-per-cpu=8G

 ### Mail to user on an event
 ### common options are FAIL, BEGIN, END, REQUEUE
 #SBATCH --mail-type=BEGIN,END,FAIL

 ### Output files
 #SBATCH -o slurm-%j.out #STDOUT
 #SBATCH -e slurm-%j.err #STDERR

 cd your_wrf_dir/run
 #srun ./real.exe
 srun ./wrf.exe
```